<style>
/* Define custom styles for headings */
.custom-heading {
    color: #4CAF50; /* Green color */
}
</style>

### <span class="custom-heading">Key Concepts Utilized in Library Management Project</span>

1. **Object-Oriented Programming (OOP):**
   - Classes (`Book`, `Member`, `Library`) encapsulate data (attributes) and behavior (methods).
   - Inheritance (`LibraryItem` as an abstract base class) promotes code reuse and hierarchy.

2. **Encapsulation:**
   - Attributes (`title`, `author`, etc.) and methods (`mark_as_borrowed()`, `add_to_history()`) are encapsulated within classes to control access and ensure data integrity.

3. **Abstraction:**
   - Abstract base class (`LibraryItem`) defines methods (`display_info()`) that are implemented in derived classes (`Book`).

4. **Polymorphism:**
   - Method overriding (`display_info()` in `Book` and `Member` classes) allows different behavior for different data types (`Book` vs. `Member`).

5. **Exception Handling:**
   - Custom exceptions (`LibraryError`) handle errors specific to library operations (e.g., adding duplicate items, borrowing non-available items).

6. **Static Methods:**
   - `welcome_message()` method in `Library` class is defined as a static method using `@staticmethod` decorator to provide a utility function.

7. **Menu-Driven Interface:**
   - Implemented using a loop (`while True`) and conditional statements (`if-elif-else`) to interactively manage library operations based on user input.

8. **Data Structures:**
   - Use of dictionaries (`self.catalog` in `Library` class) to store and manage library items efficiently by ISBN.

9. **Datetime Module:**
   - `datetime` module is used to record borrowing history (`borrow_history`) with timestamps.

10. **User Input Handling:**
    - `input()` function captures user input for interactive menu options and data entry (e.g., book details, member information).

11. **Output Formatting:**
    - Methods (`display_info()`, `display_catalog()`, `display_all_members()`) format and display information clearly for users.

12. **Loop Control:**
    - Continuous loop (`while True`) ensures the program runs until the user chooses to exit (`choice == '7'`).


In [10]:
from abc import ABC, abstractmethod
from datetime import datetime

class LibraryError(Exception):
    pass

class LibraryItem(ABC):
    def __init__(instance, title, author, isbn, genre, publication_year, copies=1):
        instance.title = title
        instance.author = author
        instance.isbn = isbn
        instance.genre = genre
        instance.publication_year = publication_year
        instance.copies = copies
        instance.available_copies = copies
        instance.status = "available"
        instance.borrow_history = []

    @abstractmethod
    def display_info(instance):
        pass

    def mark_as_borrowed(instance):
        if instance.available_copies > 0:
            instance.available_copies -= 1
            instance.status = "borrowed"
        else:
            raise LibraryError("All copies of this item are currently borrowed.")

    def mark_as_returned(instance):
        if instance.status == "borrowed":
            instance.available_copies += 1
            instance.status = "available"
        else:
            raise LibraryError("This item is not currently borrowed.")

    def add_to_history(instance, member_id):
        instance.borrow_history.append((member_id, datetime.now()))

    def display_borrow_history(instance):
        print("Borrow History:")
        for member_id, borrow_date in instance.borrow_history:
            print(f"- Member ID: {member_id}, Borrow Date: {borrow_date}")

    def __repr__(instance):
        return f"{type(instance).__name__}({instance.title}, {instance.author}, {instance.isbn}, {instance.status})"

class Book(LibraryItem):
    def __init__(instance, title, author, isbn, genre, publication_year, copies=1):
        super().__init__(title, author, isbn, genre, publication_year, copies)

    def display_info(instance):
        print(f"Title: {instance.title}")
        print(f"Author: {instance.author}")
        print(f"ISBN: {instance.isbn}")
        print(f"Genre: {instance.genre}")
        print(f"Publication Year: {instance.publication_year}")
        print(f"Copies Available: {instance.available_copies}/{instance.copies}")
        print(f"Status: {instance.status}")

class Member:
    def __init__(instance, name, member_id, member_type="regular"):
        instance.name = name
        instance.member_id = member_id
        instance.member_type = member_type
        instance.borrowed_items = []

    def borrow_item(instance, item):
        try:
            item.mark_as_borrowed()
            item.add_to_history(instance.member_id)
            instance.borrowed_items.append(item)
            return True
        except LibraryError as e:
            print(f"Error borrowing item: {e}")
            return False

    def return_item(instance, item):
        try:
            item.mark_as_returned()
            instance.borrowed_items.remove(item)
            return True
        except LibraryError as e:
            print(f"Error returning item: {e}")

    def display_info(instance):
        print(f"Name: {instance.name}")
        print(f"Member ID: {instance.member_id}")
        print(f"Member Type: {instance.member_type}")
        print("Borrowed Items:")
        if not instance.borrowed_items:
            print("No items borrowed.")
        else:
            for item in instance.borrowed_items:
                print(f"- {item.title} ({type(item).__name__})")

    def __repr__(instance):
        return f"Member({instance.name}, {instance.member_id})"

class Library:
    def __init__(instance):
        instance.catalog = {}
        instance.members = []

    def add_item(instance, item):
        if item.isbn not in instance.catalog:
            instance.catalog[item.isbn] = item
        else:
            existing_item = instance.catalog[item.isbn]
            existing_item.copies += item.copies
            existing_item.available_copies += item.available_copies
            print(f"Added {item.copies} more copies of '{item.title}' to the catalog.")

    def register_member(instance, member):
        instance.members.append(member)

    def find_item_by_isbn(instance, isbn):
        item = instance.catalog.get(isbn, None)
        if not item:
            raise LibraryError(f"Item with ISBN {isbn} not found in the catalog.")
        return item

    def find_member_by_id(instance, member_id):
        member = next((m for m in instance.members if m.member_id == member_id), None)
        if not member:
            raise LibraryError(f"Member with ID {member_id} not found.")
        return member

    def issue_item(instance, member_id, isbn):
        try:
            member = instance.find_member_by_id(member_id)
            item = instance.find_item_by_isbn(isbn)
            if isinstance(item, Book):
                return member.borrow_item(item)
            else:
                raise LibraryError("Cannot borrow this type of item.")
        except LibraryError as e:
            print(f"Error issuing item: {e}")
            return False

    def return_item(instance, member_id, isbn):
        try:
            member = instance.find_member_by_id(member_id)
            item = instance.find_item_by_isbn(isbn)
            if isinstance(item, Book):
                return member.return_item(item)
            else:
                raise LibraryError("Cannot return this type of item.")
        except LibraryError as e:
            print(f"Error returning item: {e}")

    def display_catalog(instance):
        print("Library Catalog:")
        for item in instance.catalog.values():
            item.display_info()
            print("---------------")

    def display_all_members(instance):
        print("All Members of the Library:")
        for member in instance.members:
            member.display_info()
            print("---------------")

    @staticmethod
    def welcome_message():
        print("Welcome to the Library!")


if __name__ == "__main__":
    library = Library()

    while True:
        print("\nWelcome to Liabrary Managment")
        print("\nWhat would you like to do?")
        print("1. Add a book")
        print("2. Register a member")
        print("3. Issue a book to a member")
        print("4. Return a book from a member")
        print("5. Display library catalog")
        print("6. Display all members")
        print("7. Exit")

        choice = input("Enter your choice (1-7): ")

        if choice == '1':
            # Add a book to the library
            try:
                title = input("Enter the title of the book: ")
                author = input("Enter the author of the book: ")
                isbn = input("Enter the ISBN of the book: ")
                genre = input("Enter the genre of the book: ")
                publication_year = input("Enter the publication year of the book: ")
                copies = int(input("Enter the number of copies: "))

                book = Book(title, author, isbn, genre, publication_year, copies)
                library.add_item(book)
                print(f"Book '{title}' added to the library.")
            except (LibraryError, ValueError) as e:
                print(f"Error adding book: {e}")

        elif choice == '2':
            # Register a member
            try:
                name = input("Enter the name of the member: ")
                member_id = input("Enter the member ID: ")

                member = Member(name, member_id)
                library.register_member(member)
                print(f"Member '{name}' registered.")
            except LibraryError as e:
                print(f"Error registering member: {e}")

        elif choice == '3':
            # Issue a book to a member
            try:
                member_id = input("Enter the member ID: ")
                isbn = input("Enter the ISBN of the book to issue: ")

                if library.issue_item(member_id, isbn):
                    print(f"Book issued to member {member_id}.")
                else:
                    print("Error issuing book.")
            except (LibraryError, ValueError) as e:
                print(f"Error issuing book: {e}")

        elif choice == '4':
            # Return a book from a member
            try:
                member_id = input("Enter the member ID: ")
                isbn = input("Enter the ISBN of the book to return: ")

                if library.return_item(member_id, isbn):
                    print(f"Book returned by member {member_id}.")
                else:
                    print("Error returning book.")
            except (LibraryError, ValueError) as e:
                print(f"Error returning book: {e}")

        elif choice == '5':
            # Display library catalog
            library.display_catalog()

        elif choice == '6':
            # Display all members
            library.display_all_members()

        elif choice == '7':
            # Exit the program
            print("Exiting the library management system. Goodbye!")
            break

        else:
            print("Invalid choice. Please enter a number from 1 to 7.")



Welcome to Liabrary Managment

What would you like to do?
1. Add a book
2. Register a member
3. Issue a book to a member
4. Return a book from a member
5. Display library catalog
6. Display all members
7. Exit


Enter your choice (1-7):  1
Enter the title of the book:  Data Science
Enter the author of the book:  Adham
Enter the ISBN of the book:  12
Enter the genre of the book:  2th
Enter the publication year of the book:  2024
Enter the number of copies:  50


Book 'Data Science' added to the library.

Welcome to Liabrary Managment

What would you like to do?
1. Add a book
2. Register a member
3. Issue a book to a member
4. Return a book from a member
5. Display library catalog
6. Display all members
7. Exit


Enter your choice (1-7):  1
Enter the title of the book:  ML
Enter the author of the book:  adam
Enter the ISBN of the book:  123
Enter the genre of the book:  3th
Enter the publication year of the book:  2024
Enter the number of copies:  50


Book 'ML' added to the library.

Welcome to Liabrary Managment

What would you like to do?
1. Add a book
2. Register a member
3. Issue a book to a member
4. Return a book from a member
5. Display library catalog
6. Display all members
7. Exit


...

Library Catalog:
Title: Data Science
Author: Adham
ISBN: 12
Genre: 2th
Publication Year: 2024
Copies Available: 50/50
Status: available
---------------
Title: ML
Author: adam
ISBN: 123
Genre: 3th
Publication Year: 2024
Copies Available: 49/50
Status: borrowed
---------------
Title: Deep learning 
Author: asam
ISBN: 1234
Genre: 6th
Publication Year: 2024
Copies Available: 50/50
Status: available
---------------

Welcome to Liabrary Managment

What would you like to do?
1. Add a book
2. Register a member
3. Issue a book to a member
4. Return a book from a member
5. Display library catalog
6. Display all members
7. Exit


Enter your choice (1-7):  6


All Members of the Library:
Name: Ubaid
Member ID: 10497
Member Type: regular
Borrowed Items:
No items borrowed.
---------------
Name: khan
Member ID: 15869
Member Type: regular
Borrowed Items:
- ML (Book)
---------------

Welcome to Liabrary Managment

What would you like to do?
1. Add a book
2. Register a member
3. Issue a book to a member
4. Return a book from a member
5. Display library catalog
6. Display all members
7. Exit


Enter your choice (1-7):  7


Exiting the library management system. Goodbye!
